# LISTA 4 DE VISÃO COMPUTACIONAL
# LUCAS HERON SANTOS ANCHIETA
# RUAN TENÓRIO DE MELO

In [13]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [14]:
def show_images(images, titles = None):
  n = len(images)
  plt.figure(figsize=(20, 10))
  for i in range(n):
    plt.subplot(1, n, i + 1)
    plt.imshow(images[i])
    if titles is not None:
      plt.title(titles[i])
    plt.axis('off')
  plt.show()

## Questão 1 
As fotografias em modo retrato se popularizaram nos  últimos anos. Elas consistem em segmentar foreground e background em uma fotografia, e borrar o background, simulando o efeito de uma câmera DLSR, como na imagem localizada em (./img/q1-camera.jpg).

O objetivo dessa questão é simular este efeito, usando mapas de disparidade gerados por duas imagens. Para um tutorial de como gerar mapas de disparidade usando OpenCV, veja https://docs.opencv.org/4.7.0/dd/d53/tutorial_py_depthmap.html. Após a aquisição do mapa de disparidade, você deve usá-lo adequadamente para detectar o foreground, aplicar um filtro gaussiano para borrar apenas o background, e combinar as duas partes para obter um efeito como na figura acima. Experimente seu algoritmo em três pares de imagens, em cenários diferentes, adquiridas através de pequenas variações de ponto de vista da câmera.

In [15]:
imgL = cv2.imread('./img/controller-left.jpg', 0)
imgR = cv2.imread('./img/controller-right.jpg', 0)

In [16]:
ply_header = '''ply
format ascii 1.0
element vertex %(vert_num)d
property float x
property float y
property float z
property uchar red
property uchar green
property uchar blue
end_header
'''

def write_ply(fn, verts, colors):
    verts = verts.reshape(-1, 3)
    colors = colors.reshape(-1, 3)
    verts = np.hstack([verts, colors])
    with open(fn, 'wb') as f:
        f.write((ply_header % dict(vert_num=len(verts))).encode('utf-8'))
        np.savetxt(f, verts, fmt='%f %f %f %d %d %d ')

In [18]:
print('loading images...')
imgL = cv2.pyrDown(cv2.imread("./img/controller-left.jpg")) 
imgR = cv2.pyrDown(cv2.imread("./img/controller-right.jpg"))

# disparity range is tuned for 'aloe' image pair
window_size = 3
min_disp = 16
num_disp = 112-min_disp
stereo = cv2.StereoSGBM_create(minDisparity = min_disp,
    numDisparities = num_disp,
    blockSize = 16,
    P1 = 8*3*window_size**2,
    P2 = 32*3*window_size**2,
    disp12MaxDiff = 1,
    uniquenessRatio = 10,
    speckleWindowSize = 100,
    speckleRange = 32
)

print('computing disparity...')
disp = stereo.compute(imgR, imgL).astype(np.float32) / 16.0

print('generating 3d point cloud...',)
h, w = imgL.shape[:2]
f = 0.8*w                          # guess for focal length
Q = np.float32([[1, 0, 0, -0.5*w],
                [0,-1, 0,  0.5*h], # turn points 180 deg around x-axis,
                [0, 0, 0,     -f], # so that y-axis looks up
                [0, 0, 1,      0]])
points = cv2.reprojectImageTo3D(disp, Q)
colors = cv2.cvtColor(imgL, cv2.COLOR_BGR2RGB)
mask = disp > disp.min()
out_points = points[mask]
out_colors = colors[mask]
out_fn = 'out.ply'
write_ply(out_fn, out_points, out_colors)
print('%s saved' % out_fn)

cv2.imshow('left', imgL)
cv2.imshow('disparity', (disp-min_disp)/num_disp)
cv2.waitKey()

print('Done')
cv2.destroyAllWindows()

loading images...
computing disparity...
generating 3d point cloud...
out.ply saved
Done


## Questão 2 
Leia o seguinte tutorial sobre reconstrução 3d, até a etapa de geração do arquivo PLY: https://medium.com/analytics-vidhya/depth-sensing-and-3d-reconstruction-512ed121aa60

a) Experimente o código disponibilizado no tutorial em três exemplos distintos, gerando a nuvem de pontos e visualizando com o Meshlab.

b) Descreva, com suas palavras, as etapas realizadas desde o fornecimento do par de imagens de entrada, até a geração da nuvem de pontos em formato PLY.

## Questão 3 
Usando a plataforma Kaggle, pesquise um base supervisionada de imagens que possa ser usada para treinar um classificador binário de imagens. Esta base deve conter exemplos de duas classes. Considere 70% das amostras para treino, e 30% para teste (método holdout).

a) Usando OpenCV, extraia descritores HOG (Histogram of Gradients), e treine um classificador SVM. Exiba a acurácia atingida no conjunto de teste e a matriz de confusão. Exiba exemplos. Obs.: note que essa abordagem é semelhante ao detector Dalal-Triggs, porém aqui estamos usando-a para classificação de imagens. 

b) Agora o objetivo é treinar uma CNN. Usando a abordagem de transferência de aprendizado (transfer learning), realize treinamentos usando modelos VGG16, ResNet50, e MobileNetV2 pré-treinados. Compare os resultados usando acurácias e matrizes de confusão. Exiba exemplos de acerto e erro.

## Questão 4 
O objetivo dessa questão é criar um contador de veículos em vídeos de uma rodovia. Você deve experimentar um modelo pré-treinado da YOLO, e um Single Shot MultiBox Detection (SSD).

a) Estude a SSD, e descreva suas principais diferenças em relação à YOLO.

b) Para cada um dos dois modelos, realize as detecções de veículos em cada frame do vídeo em anexo (classroom). Gere os vídeos com as bounding boxes detectadas.

c) Para cada um dos dois modelos, plote um gráfico onde o eixo horizontal é o tempo (quadro do vídeo), e o vertical é a quantidade de carros detectada pelo modelo.